In [1]:
import numpy as np
import tensorflow as tf
from scipy import spatial
from scipy import stats
from gensim.models import word2vec

# Import pandas
import pandas as pd
from pandas import Series, DataFrame

# Import util
import time
import re
import sys
import gc

# Self define module
from mini_batch_helper import MiniBatchCorpus

Using TensorFlow backend.


### Loading corpus and forming dict

In [2]:
corpus_fnames = [
    'datas/training_data/no_TC_下課花路米.txt',
    'datas/training_data/no_TC_誰來晚餐.txt',
    'datas/training_data/no_TC_公視藝文大道.txt',
    'datas/training_data/no_TC_成語賽恩思.txt',
    'datas/training_data/no_TC_我的這一班.txt',
    'datas/training_data/no_TC_流言追追追.txt',
    'datas/training_data/no_TC_人生劇展.txt',
    'datas/training_data/no_TC_聽聽看.txt',
]
sample_rate_on_training_datas = 0.3
valid_cp_num_of_each = 1

def word_tok_lst_2_ch_lst(s):
    return [ch.strip() for word in s for ch in word if ch.strip() != '']

def corpus_extract_sentence(now_corpus):
    return [[ch for ch in word_tok_lst_2_ch_lst(s)] for line in now_corpus for s in line.strip().split('\t')]

corpus = []
corpus_valid = []
for fname in corpus_fnames:
    with open(fname, 'r') as f:
        now_corpus = np.array([line for line in f])
        now_corpus_valid = now_corpus[:valid_cp_num_of_each]
        now_corpus = now_corpus[valid_cp_num_of_each:]
        if sample_rate_on_training_datas < 1:
            sample_num = int(max(len(now_corpus)*sample_rate_on_training_datas, 5))
            rnd_idx = np.arange(len(now_corpus))
            np.random.shuffle(rnd_idx)
            now_corpus = now_corpus[rnd_idx[:sample_num]]
        
        corpus.append(corpus_extract_sentence(now_corpus))
        corpus_valid.append(corpus_extract_sentence(now_corpus_valid))

with open('datas/dict/id2ch.txt') as f:
    id2ch = f.read().strip().split()
ch2id = dict([(ch, i) for i, ch in enumerate(id2ch)])

In [3]:
corpus_id = [[[ch2id[ch] for ch in s if ch in ch2id] for s in cp] for cp in corpus]
corpus_valid_id = [[[ch2id[ch] for ch in s if ch in ch2id] for s in cp] for cp in corpus_valid]

In [4]:
# Statistic sentence length
Series([len(s) for cp in corpus for s in cp]).describe()

count    703973.000000
mean          7.702720
std           3.529458
min           1.000000
25%           5.000000
50%           8.000000
75%          10.000000
max          25.000000
dtype: float64

In [5]:
voc_size = len(ch2id)
emb_size = 200
pad_word_id = ch2id['<eos>']
max_seq_len = np.max([len(s) for cp in corpus_id for s in cp])

print('%20s: %d' % ('pad_word_id', pad_word_id))
print('%20s: %d' % ('max_seq_len', max_seq_len))

         pad_word_id: 1894
         max_seq_len: 25


In [6]:
train_data_loader = MiniBatchCorpus(corpus_id)
valid_data_loader = MiniBatchCorpus(corpus_valid_id)
print('train datas num:', train_data_loader.data_num)
print('valid datas num:', valid_data_loader.data_num)

train datas num: 1407930
valid datas num: 16432


### Define model

In [7]:
# Input
x1 = tf.placeholder(tf.int32, [None, None])
x2 = tf.placeholder(tf.int32, [None, None])
y = tf.placeholder(tf.float64, [None])

# Embedding layer
embeddings_W = tf.Variable(tf.truncated_normal([voc_size, emb_size], stddev=0.01, dtype=tf.float64))

In [8]:
def sentence_embedding(xs):
    xs_mask = 1 - tf.to_double(tf.equal(xs, pad_word_id))
    xs_len = tf.reduce_sum(xs_mask, axis=1)
    xs_embedded = tf.gather(embeddings_W, xs)
    xs_center = tf.reduce_sum(xs_embedded, axis=1) / tf.reshape(tf.to_double(xs_len)+1e-6, [-1, 1])
    return xs_center

In [9]:
x1_center = sentence_embedding(x1)
x2_center = sentence_embedding(x2)
W = tf.Variable(tf.truncated_normal([emb_size, emb_size], stddev=0.01, dtype=tf.float64))
tf_score = tf.reduce_sum((x2_center * (x1_center @ W)), axis=1)

In [10]:
tf_prob = tf.sigmoid(tf_score)
tf_correct = tf.reduce_sum(tf.cast(
    (tf.equal(y, 1) & tf.greater_equal(tf_prob, 0.5)) | (tf.equal(y, 0) & tf.less(tf_prob, 0.5)),
    tf.int32
))

----------------------------------
## Training

In [11]:
reg = tf.nn.l2_loss(W) * 0
cost = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=tf_score)) + reg
optimizer = tf.train.AdamOptimizer(1e-3)
gvs = optimizer.compute_gradients(cost)
capped_gvs = [(tf.clip_by_norm(grad, 0.2), var) for grad, var in gvs]
train_step = optimizer.apply_gradients(capped_gvs)

In [12]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
def eval_valid_loss():
    valid_loss = 0
    valid_acc = 0
    valid_batch = 1024
    batch_num = valid_data_loader.data_num // valid_batch
    for i in range(batch_num):
        b_x1, b_x2, b_y = valid_data_loader.next_batch(valid_batch, max_seq_len, pad_word_id)
        now_loss, now_correct = sess.run([cost, tf_correct], {x1: b_x1, x2: b_x2, y: b_y})
        valid_loss += now_loss / (batch_num * valid_batch)
        valid_acc += now_correct / (batch_num * valid_batch)
    return valid_loss, valid_acc

In [14]:
batch_size = 256
epoch_num = 10
log_interval = 500
save_interval = 10000

last_epoch = None
train_batch_loss = 0
start_time = time.time()
for i_batch in range(epoch_num * train_data_loader.data_num // batch_size):
    epoch = i_batch // (train_data_loader.data_num // batch_size)
    if last_epoch is None or last_epoch != epoch:
        last_epoch = epoch
        print('Start epoch %d' % (epoch))
    
    b_x1, b_x2, b_y = train_data_loader.next_batch(batch_size, max_seq_len, pad_word_id)
    _, now_loss = sess.run([train_step, cost], {x1: b_x1, x2: b_x2, y: b_y})
    train_batch_loss += now_loss / (log_interval * batch_size)
    if (i_batch+1) % log_interval == 0:
        valid_loss, valid_acc = eval_valid_loss()
        print('train batch loss %10f / valid loss %10f / valid acc %10f / elapsed time %.f' % (
            train_batch_loss, valid_loss, valid_acc, time.time()-start_time), flush=True)
        train_batch_loss = 0
    if save_interval is not None and (i_batch+1) % save_interval == 0:
        saver.save(sess, 'models/Attack-sentence-embedding/s_emb', global_step=i_batch+1)
        print('model saved (latest)', flush=True)

saver.save(sess, 'models/Attack-sentence-embedding/s_emb_final')

Start epoch 0
train batch loss   0.692405 / valid loss   0.686703 / valid acc   0.552917 / elapsed time 8
train batch loss   0.680337 / valid loss   0.675494 / valid acc   0.571350 / elapsed time 17
train batch loss   0.676207 / valid loss   0.672791 / valid acc   0.572693 / elapsed time 25
train batch loss   0.673921 / valid loss   0.672222 / valid acc   0.578247 / elapsed time 33
train batch loss   0.672471 / valid loss   0.670038 / valid acc   0.580872 / elapsed time 42
model saved (latest)
train batch loss   0.671657 / valid loss   0.672461 / valid acc   0.590576 / elapsed time 50
train batch loss   0.669507 / valid loss   0.666339 / valid acc   0.594482 / elapsed time 58
train batch loss   0.661585 / valid loss   0.669014 / valid acc   0.601746 / elapsed time 67


KeyboardInterrupt: 

------------------------------
## Evaluating

In [ ]:
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, 'models/Attack-sentence-embedding/best/model')

In [ ]:
sample = pd.read_csv('datas/sample_test_data.txt')

sample_x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.dialogue.values]
sample_x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.options.values]
sample_y = sample.answer.values

def str_2_idlst(s):
    idlst = [ch2id[ch] for ch in s if ch in ch2id]
    return idlst

sample_id1 = []
for q in sample_x1:
    sample_id1.append([])
    for qq in q:
        sample_id1[-1].extend(str_2_idlst(qq))
sample_id1 = np.array([s + [pad_word_id] * (max_seq_len - len(s)) for s in sample_id1])

sample_id2 = []
for rs in sample_x2:
    sample_id2.append([])
    for r in rs:
        sample_id2[-1].append(str_2_idlst(r))
sample_id2 = np.array([[s + [pad_word_id] * (max_seq_len - len(s)) for s in rs] for rs in sample_id2])

In [ ]:
scores = sess.run(tf_score, {x1: np.repeat(sample_id1, 6, axis=0), x2: sample_id2.reshape(-1, max_seq_len)})
scores = scores.reshape(-1, 6)

In [ ]:
np.sum(np.argmax(scores, axis=1) == sample_y)